In [ ]:
import functools as ft
from matplotlib import pyplot as plt
import matplotlib as mpl
import networkx as nx
import numpy as np
import pickle

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS

from cost_based_selection.preprocessing_utils import load_simulations
from cost_based_selection.summaries import compute_summaries

from scipy.spatial.distance import pdist
from scipy.stats import pearsonr, ks_2samp

mpl.style.use("../scrartcl.mplstyle")

In [ ]:
# Load the feature rankings.
with open("../workspace/dmX/rankings/train/JMI/penalty-1.0.pkl", "rb") as fp:
    result = pickle.load(fp)
    
num_features = 5
ranking = result["ranking"][:num_features]

In [ ]:
# Get the simulations with the right number of features.
keys = ["ito2001", "yu2008", "uetz2000"]
simulations_by_key = {
    key: load_simulations(f"../workspace/dmX/simulations/yeast/{key}/*.pkl")
    for key in keys
}

In [ ]:
# Load the protein interaction graphs and compute summaries.
graphs_by_key = {}
summaries_by_key = {}

for key, simulations in simulations_by_key.items():
    graph = nx.Graph()
    for row in np.loadtxt(f"../workspace/dmX/data/{key}.txt", dtype=str):
        i, j, *_ = row
        if i != j:
            graph.add_edge(i, j)
    graphs_by_key[key] = graph

    summaries, *_ = compute_summaries(graph)
    X = np.atleast_2d([summaries[key] for key in simulations["features"]])
    summaries_by_key[key] = X

In [ ]:
# Train the nearest neighbor classifiers.
for key, summaries in summaries_by_key.items():
    simulations = simulations_by_key[key]
    
    model_cls = ft.partial(KNeighborsClassifier, 10)
    pipeline = make_pipeline(StandardScaler(), model_cls())
    pipeline.fit(simulations["X"][:, ranking], simulations["y"])
    print(key, pipeline.predict(summaries[:, ranking]), pipeline.predict_proba(summaries[:, ranking]))

In [ ]:
# Learn the embeddings with multidimensional scaling.
num_points = 1000
embeddings_by_key = {}

for key, simulations in simulations_by_key.items():
    mds = MDS(random_state=0)
    X = simulations["X"][:num_points]
    # Prepend the observed data so we can embed together.
    X = np.vstack([summaries_by_key[key], X])
    y = simulations["y"][:num_points]

    embedding_pipeline = make_pipeline(StandardScaler(), mds)
    embedding_pipeline.fit(X[:, ranking])
    embeddings_by_key[key] = mds.embedding_
    
    embedding_dist = pdist(mds.embedding_)
    X_dist = pdist(X[:, ranking])
    print(key, pearsonr(embedding_dist, X_dist))

    # Show the variance eigenvalues.

In [ ]:
# Show variances by model (we expect variances to correlate with the 
# "spread" of points obtained by MDS).
for key, simulations in simulations_by_key.items():
    X = simulations["X"][:, ranking]
    y = simulations["y"]
    for value in np.unique(y):
        cov = np.cov(X[y == value], rowvar=False)
        eigvals = np.linalg.eigvals(cov)
        print(f"{key}, {value}: {eigvals.sum() ** .5:.1f}")

In [ ]:
# fig, axes = plt.subplots(2, 2, sharex=True, sharey=True)
fig = plt.figure()
gs = fig.add_gridspec(2, 2)

ax = None
axes = []
for i in range(2):
    for j in range(2):
        # Don't share the last axis.
        if i == 1 and j == 1:
            ax = None
        ax = fig.add_subplot(gs[i, j], sharex=ax, sharey=ax)
        axes.append(ax)

signs_by_key = {
    "yu2008": [1, -1],
    "uetz2000": -1,
}

for ax, (key, embedding) in zip(axes, embeddings_by_key.items()):
    c = ["C0" if x == "dmr" else "C1" for x in simulations_by_key[key]["y"][:num_points]]
    X = embedding
    
    # Rotate to align.
    y = simulations_by_key[key]["y"][:num_points]
    x = X[1:][y == "dmc"]
    cov = np.cov(x.T)
    eigval, eigvecs = np.linalg.eigh(cov)
    eigvec = eigvecs[:, np.argmax(eigval)]
    angle = np.arctan2(*eigvec[::-1])
    rotation = np.asarray([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle), np.cos(angle)],
    ])
    X = (X @ rotation) * signs_by_key.get(key, 1)
    
    ax.scatter(*X[1:].T, c=c, alpha=.5)
    ax.scatter(*X[0], marker='X', color='k').set_edgecolor('w')
    ax.set_aspect("equal")
    ax.set_axis_off()
    
    print(key)
    
    
for ax, label in zip(axes, "abc"):
    ax.text(0.05, 0.95, f"({label})", transform=ax.transAxes, va='top')
    
# Let's make sure we get the feature we expect.
key = "ito2001"
simulations = simulations_by_key[key]
feature_name = simulations["features"][ranking[0]]
assert feature_name == 'transitivity'

ax = axes[-1]
x = simulations["X"][:, ranking[0]]
lims = x.min(), x.max()
y = simulations["y"]
for z in ["dmr", "dmc"]:
    ax.hist(x[y == z], density=True, bins=20, label=z.upper(), range=lims)
print(ks_2samp(x[y == "dmr"], x[y == "dmc"]))
    
ax.set_xlabel("Transitivity $T$")
ax.set_ylabel("$p(T)$")
ax.text(0.95, 0.05, "(d)", transform=ax.transAxes, va='bottom', ha='right')
ax.axvline(summaries_by_key[key][:, ranking[0]], color='k', ls=':',
           label="observed (Ito et al. 2001)")

ax.legend(loc="best")
fig.tight_layout()
fig.savefig("yeast.pdf")